<a href="https://colab.research.google.com/github/JaeyeongLee1/DSAIL_summer_internship/blob/main/FM_pytorch%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import sklearn
from sklearn import datasets
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

#breast cancer prediction model with Factorization Machine

#preprocessing
cancer = sklearn.datasets.load_breast_cancer()
scaler = MinMaxScaler()
data = scaler.fit_transform(cancer.data)
y = cancer.target
data = pd.DataFrame(data = data, columns = cancer.feature_names)

n = data.shape[1]
k = 10
X = torch.FloatTensor(data.values)
Y = torch.FloatTensor(y).unsqueeze(1)

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

class FM(nn.Module):
  def __init__(self,n,k):
    super().__init__()
    self.w0 = nn.Parameter(torch.zeros(1))
    self.W = nn.Parameter(torch.zeros((n,1)))
    self.V = nn.Parameter(1e-6*torch.rand((n,k)))
    self.n = n
    self.k = k

  def forward(self,x):
    w0 = self.w0
    W = self.W
    V = self.V
    n = self.n
    k = self.k
    y_hat = w0 + x.matmul(W)
    for user in range(x.shape[0]):
      for f in range(k):
        y_hat[user] += 0.5*((V.T[f].dot(x[user]))**2 - (V.T[f]**2).dot(x[user]**2))
    return torch.sigmoid(y_hat)

  def accuracy(self,x,y):
    predictions = self.forward(x)
    count = 0
    for i in range(x.shape[0]):
      if predictions[i]>=0.5:
        if y[i] == 1:
          count += 1
      else:
        if y[i] == 0:
          count += 1
    return count/rows

  def optimize(self,x,y,lr,epochs,batch_size):
    dataset = TensorDataset(X,Y)
    dataloader = DataLoader(dataset, batch_size = batch_size,shuffle = True)
    optimizer = torch.optim.SGD(self.parameters(),lr = lr)
    for i in range(epochs):
      for batchid,samples in enumerate(dataloader):
        X_train,Y_train = samples
        optimizer.zero_grad()
        cost = F.binary_cross_entropy(model(X_train),Y_train)
        cost.backward()
        optimizer.step()

dataset = TensorDataset(X,Y.unsqueeze(1))
dataloader = DataLoader(dataset,batch_size = 100,shuffle = True)
model = FM(n,k)
model.optimize(X,Y,lr = 0.01, epochs = 200, batch_size = 100)

In [15]:
model.accuracy(X,Y)

0.9086115992970123